- 语言内部使用 iter(...) 内置函数处理可迭代对象的方式
- 如何使用 Python 实现经典的迭代器模式
- 详细说明生成器函数的工作原理
- 如何使用生成器函数或生成器表达式代替经典的迭代器
- 如何使用标准库中通用的生成器函数
- 如何使用 yield from 语句合并生成器
- 案例分析:在一个数据库转换工具中使用生成器函数处理大型数据集
- 为什么生成器和协程看似相同，实则差别很大，不能混淆

- 什么是迭代器
### 迭代器
  迭代器是这样的对象:实现了无参数的 \_\_next__ 方法，返回序
列中的下一个元素;如果没有元素了，那么抛出 StopIteration 异
常。Python 中的迭代器还实现了 \_\_iter__ 方法，因此迭代器也可以迭代。

主要用在迭代操作中，如for循环

In [52]:
# 实现迭代器（迭代器是对象，即类）
# 迭代器原理

# 斐波那契数列
class Fib:

    def __init__(self, n):
        self._prev = 0
        self._cur = 1
        self._n = n

    def __iter__(self):
        return self # 特点一，迭代时返回自己，而非数据

    # 特点二：实现next，并只返回这一个元素（同一时间只返回一个元素，因此占用内存更少！
    def __next__(self):
        if self._n > 0:
            value = self._cur
            self._prev = self._cur
            self._cur = self._prev + self._cur
            self._n -= 1
            return value
        else:
            raise StopIteration()

In [56]:
f =Fib(5)
for v in f:
    print(v)

1
2
4
8
16


### 什么是生成器
生成器是一种函数（迭代器是一个对象），使用yield关键字作为返回操作（普通的函数使用
return），返回的类型是生成器对象。生成器对象本质就是迭代器，没有作用区别，只是前者
是自动生成的，语法上更加简洁。

In [57]:
# 生成器有两种方式，一份是生成器函数，一个是生成器表达式
# 斐波那契数列 生成器函数：
def fib(n):
    prev, cur = 0, 1
    while n > 0:
        yield cur # 此处返回一个生成器，但函数还会继续运行！
        prev, cur = cur, cur + prev
        n -= 1

for v in fib(6):
    print(v)

1
1
2
3
5
8


In [67]:
# 生成器表达式(用小括号包围起来）：
a = 1
for v in (x for x in range(6)):
    print(v)

g = (x for x in range(6))
g # generator

# 下面是列表推导式
for s in [x*2 for x in range(6)]:
    print(s)

l = [x*2 for x in range(6)]
l # list

0
1
2
3
4
5
0
2
4
6
8
10


[0, 2, 4, 6, 8, 10]

- 详细说明生成器函数的工作原理
### 生成器函数的工作原理
只要 Python 函数的定义体中有 yield 关键字，该函数就是生成器函 数。
调用生成器函数时，会返回一个生成器对象。也就是说，生成器函数是生成器工厂。

In [41]:
# 惰性实现
import re
import reprlib

RE_WORD = re.compile("/w+")

class Sentence:
    def __init__(self, text):
        self._text = text

    def __repr__(self):
        return 'Sentence(%s)' % reprlib.repr(self._text)

    def __iter__(self):
        for match in RE_WORD.finditer(self._text):
            yield match.group()

In [42]:
s = Sentence('good way to die!')
s

Sentence('good way to die!')

In [43]:
iter(s)

<generator object Sentence.__iter__ at 0x7fa0ec15eac0>

In [51]:
a = RE_WORD.finditer('abc')

for aa in a:
    g = aa.group()

SyntaxError: 'yield' outside function (<ipython-input-51-4402624b3873>, line 5)

In [ ]:
b = RE_WORD.findall('abc')

### 可迭代对象不能是迭代器：
为了“支持多种遍历”，必须能从同一个可迭代的实例中获取多个独立的 迭代器，而且各个迭代器要能维护自身的内部状态，因此这一模式正确
的实现方式是，每次调用 iter(my_iterable) 都新建一个独立的迭 代器。这就是为什么这个示例需要定义 SentenceIterator 类。


In [64]:
# 学会使用迭代器来代替生成列表
# 生成器版本
def numbers(n):
    for n in range(n):
        yield n*2

for x in numbers(5):
    print(x)

0
2
4
6
8


In [65]:
# 普通版本
def nnumbers(n):
    res = []
    for x in range(n):
        res.append(x*2)
    return res
for x in nnumbers(5):
    print(x)


0
2
4
6
8


### 注意一个生成器只能迭代一次！

In [73]:
# 注意一个生成器只能迭代一次！
g = (x for x in range(6))
next(g)

0

In [74]:
next(g)

1

In [75]:
next(g)

2

In [76]:
a = list(g)
a

[3, 4, 5]